## 1. Portfolio Analysis

Calculate these key metrics using the data above:
- Portfolio total return for the period
- Top 2-3 contributing positions (by absolute dollar impact)
- Key detracting position(s) (by absolute dollar impact)
- Sector performance breakdown

### Top Contributors, Detractors and Sector Breakdown:

In [1]:
from tabulate import tabulate
import numpy as np
from portfolio_analyzer import PortfolioAnalyzer
analyzer = PortfolioAnalyzer("data/PortfolioPositions_Updated.xlsx")
perf = analyzer.calculate_performance()

# Prepare data for the table
overview_data = [
    ["Portfolio Start Value (USD)", f"{perf['portfolio_start_value_usd']:,.2f}"], # type : ignore 
    ["Portfolio End Value (USD)", f"{perf['portfolio_end_value_usd']:,.2f}"],
    ["Total PnL (USD)", f"{perf['total_pnl_usd']:,.2f}"],
    ["Total Return (%)", f"{perf['total_return_pct'] * 100:,.2f}%"]
]

# Print the table
print(tabulate(overview_data, headers=["Metric", "Value"], tablefmt="grid"))

+-----------------------------+--------------+
| Metric                      | Value        |
+=============================+==============+
| Portfolio Start Value (USD) | 1,662,200.97 |
+-----------------------------+--------------+
| Portfolio End Value (USD)   | 1,808,580.80 |
+-----------------------------+--------------+
| Total PnL (USD)             | 146,379.83   |
+-----------------------------+--------------+
| Total Return (%)            | 8.81%        |
+-----------------------------+--------------+


### 1.1 Top Contributors:

In [2]:
display(perf["top_contributors"]) # type: ignore

,ticker,name,currency,sector,start_value_usd,pnl_usd,local_return,fx_return,total_return_usd
0,NESN.SW,NESTLE SA-REG,CHF,Consumer Staples,109605.00,90670.00,0.096346,0.666667,0.827243
1,OR.PA,L'OREAL,EUR,Consumer Staples,294800.00,49924.00,0.016825,0.150000,0.169349
2,CAT,CATERPILLAR INC,USD,Industrials,371110.74,36618.66,0.098673,0.000000,0.098673


### 1.2 Top Detractors

In [3]:
display(perf["top_detractors"]) # type: ignore

,ticker,name,currency,sector,start_value_usd,pnl_usd,local_return,fx_return,total_return_usd
0,AZN.L,ASTRAZENECA PLC,GBP,Health Care,176116.500,-23852.700,0.120730,-0.228571,-0.135437
1,HD,HOME DEPOT INC,USD,Consumer Discretionary,291736.800,-13327.200,-0.045682,0.000000,-0.045682
2,JPM,JPMORGAN CHASE & CO,USD,Financials,235427.925,-10097.925,-0.042892,0.000000,-0.042892


### 1.3 Sector Performance Breakdown:

In [4]:
display(perf["sector_breakdown"]) # type: ignore

,sector,start_value_usd,end_value_usd,pnl_usd,sector_return_pct,portfolio_weight,end_portfolio_weight,weight_change_pct,target_weights
4,Consumer Staples,404405.000,544999.0,140594.000,0.347656,0.243295,0.301341,0.058046,0.24
0,Industrials,371110.740,407729.4,36618.660,0.098673,0.223265,0.225442,0.002177,0.22
1,Information Technology,183404.000,199849.0,16445.000,0.089665,0.110338,0.110500,0.000162,0.11
3,Financials,235427.925,225330.0,-10097.925,-0.042892,0.141636,0.124589,-0.017047,0.14
2,Consumer Discretionary,291736.800,278409.6,-13327.200,-0.045682,0.175512,0.153938,-0.021574,0.18
5,Health Care,176116.500,152263.8,-23852.700,-0.135437,0.105954,0.084190,-0.021764,0.11


### 1.4 Test Results against expected outputs:

In [5]:
from tests import run_tests
run_tests();

Top Contributors: ✅ match
Top Detractors: ✅ match
Sector Breakdown: ✅ match

----------------------------------------
✅ All calculated values match the hand-computed reference solutions within tolerance.
The portfolio performance calculations are consistent and numerically validated.
----------------------------------------


## 2. Market Context Retrieval (RAG Implementation)

Research the top 2-3 contributing positions using feeds from external data provider such as Yahoo Finance. Yahoo Finance is a free news provider which can be accessed in Python using yfinance (feel free to use alternative data source):
- Identify the largest contributors from your portfolio analysis
- Focus on major corporate events during September 30 - October 24, 2025
- Research company-specific news explaining performance drivers

### 2.1 Fetch News Articles for Top Contributors:

In [6]:
# from get_news import fetch_and_save_news

# # Run them individually to not hit rate limits

# fetch_and_save_news(
#     #tickers=["NESN.SW", "OR.PA", "CAT", "AZN.L", "HD", "JPM"],
#     #tickers=["CAT", "AZN.L", "HD", "JPM"], 
#     date_from="2025-09-30",
#     date_to="2025-10-24"
# )

### 2.2 Statistics Overview
Here are the statistics for the news articles fetched for each ticker. The reason we save the statistics per ticker and not combined is that we hit a rate limit on the news API, so we fetch articles for each ticker separately with alternating API keys. This is just a workaround and would not be the case in a production environment. For OR.PA and NESN.SW I populated the news articles manually as they were not available via the API as you can see below, the 1 article saved for each ticker is just the fallback article.

In [7]:
# Combine all individual ticker statistics into a single CSV file
import os
import pandas as pd


stats_dir = "data/news/fetch_statistics"
all_stats = []

for file_name in os.listdir(stats_dir):
    if file_name.endswith(".csv") and "fetch_stats" in file_name and file_name != "combined_news_fetch_statistics.csv":
        file_path = os.path.join(stats_dir, file_name)
        df = pd.read_csv(file_path)
        all_stats.append(df)


combined_stats = pd.concat(all_stats, ignore_index=True)
combined_stats.to_csv(os.path.join(stats_dir, "combined_news_fetch_statistics.csv"), index=False)
print(f"Combined {len(all_stats)} statistics files into combined_news_fetch_statistics.csv")
    
# Display the combined statistics
combined_stats

Combined 6 statistics files into combined_news_fetch_statistics.csv


,symbol,date_from,date_to,api_source,total_found,fetch_attempted,fetch_success,fetch_blocked,fetch_error,filtered_word_count,filtered_relevance,articles_saved,fallback_used,min_relevance,max_relevance,avg_relevance
0,HD,2025-09-30,2025-10-24,finnhub,86,86,64,21,1,10,22,32,False,0.12,1.00,0.420
1,NESN.SW,2025-09-30,2025-10-24,marketaux,3,3,1,2,0,0,1,1,True,NaN,NaN,NaN
2,OR.PA,2025-09-30,2025-10-24,marketaux,1,1,0,1,0,0,0,1,True,NaN,NaN,NaN
3,CAT,2025-09-30,2025-10-24,finnhub,103,103,70,31,2,16,24,30,False,0.12,1.00,0.508
4,JPM,2025-09-30,2025-10-24,finnhub,226,226,183,41,2,56,49,78,False,0.12,1.00,0.473
5,AZN.L,2025-09-30,2025-10-24,marketaux,20,20,17,2,1,0,6,11,False,0.20,0.52,0.378


## 3. Narrative Generation
Generate a 6-10 sentence narrative that includes:
1. Overall portfolio performance summary
2. Top contributor with market context explaining why it performed well
3. Second key contributor with context
4. Key detractor with explanation
5. Portfolio themes and sector performance summary

In [8]:
# narrative = analyzer.generate_narrative()
# print(narrative)

Already generated narrative is displayed below:

In [9]:
with open("results/portfolio_narrative.txt", "r") as file:
    content = file.read()
    print(content)

The portfolio gained 8.81% from September 30 to October 24, 2025, generating $146,380 in profit.
Nestlé SA-REG (NESN.SW) was the largest contributor, advancing 82.72% and adding $90,670 to portfolio value after the company announced roughly 16,000 job cuts and raised its cost-savings target to CHF 3 billion alongside better-than-expected Q3 organic growth, which sent shares sharply higher; the move comprised a 9.63% local gain and a substantial 66.7% FX tailwind [2][3].
L’Oréal (OR.PA) was also a key contributor, rising 16.93% and adding $49,924 as nine-month sales reached €32.8 billion with 3.4% like-for-like growth and the company agreed to acquire Kering’s beauty unit, including Creed and long-term licences for Gucci, Balenciaga and Bottega Veneta; performance reflected a 1.68% local return and a 15.0% FX boost [4][5][6].
Caterpillar Inc (CAT) gained 9.87%, contributing $36,619, as investors looked ahead to the upcoming Q3 print and the newly announced Nov. 4 Investor Day webcast ou